In [2]:
%defaultDatasource jdbc:h2:mem:db

# Analisando o Uso de Medicamentos
* Esta tarefa analisará os dados conforme três abordagens: análise do comportamento estatístico; análise de correlação entre pares; análise de correlação de variáveis.
* A partir dos arquivos CSV que estão no diretórios /data/nhanes considere as seguintes tabelas SQL.

In [3]:
CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

CREATE TABLE Medicamento (
  Codigo VARCHAR(6) NOT NULL,
  NomeMedicamento VARCHAR(100) NOT NULL,
  PRIMARY KEY(Codigo)
) AS SELECT
    Codigo,
    NomeMedicamento
FROM CSVREAD('../../../data/nhanes/medications-drug.csv');

CREATE TABLE UsoMedicamento (
  IdPessoa VARCHAR(5) NOT NULL,
  CodMedicamento VARCHAR(6) NOT NULL,
  DiasUso INTEGER,
  FOREIGN KEY(IdPessoa)
    REFERENCES Pessoa(Id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(CodMedicamento)
    REFERENCES Medicamento(Codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    IdPessoa,
    CodMedicamento,
    DiasUso
FROM CSVREAD('../../../data/nhanes/medications-use.csv');

In [4]:
SELECT * FROM Pessoa;
SELECT * FROM Medicamento;
SELECT * FROM UsoMedicamento;

# I. Análise de comportamento estatístico

## 1) Contabilizando o uso de medicamentos

* Liste o nome de todos os medicamentos e a quantidade de pessoas que usa cada um deles.

In [ ]:
SELECT M.NomeMedicamento, count(UM.IdPessoa)
FROM UsoMedicamento UM, Medicamento M
WHERE UM.CodMedicamento = M.Codigo
GROUP BY UM.CodMedicamento

## 2) Medicamento mais usado com VIEW

* Informe o código do medicamento mais usado: fazendo uso de VIEW.

In [ ]:
DROP VIEW IF EXISTS MaxUsado;

CREATE VIEW MaxUsado AS
SELECT UM.CodMedicamento, count(UM.DiasUso) AS Numero
FROM UsoMedicamento UM
GROUP BY UM.CodMedicamento
ORDER BY Numero DESC limit 1;

SELECT * FROM MaxUsado;

## 3) Medicamento mais usado sem VIEW

* Informe o código do medicamento mais usado: sem fazer uso de VIEW.

In [ ]:
SELECT UM.CodMedicamento, count(UM.DiasUso) AS Numero
FROM UsoMedicamento UM
GROUP BY UM.CodMedicamento
ORDER BY Numero DESC limit 1;

## 4) Nome do medicamento mais usado
* Informe o nome do medicamento mais usado (uso de VIEW é opcional).

In [ ]:
SELECT M.NomeMedicamento, MU.Numero
FROM Medicamento M, MaxUsado MU
WHERE M.Codigo = MU.CodMedicamento

## 5) Contabilizando quanto as pessoas usam de medicamento

* Informe o número médio de uso de medicamento por pessoa.

In [ ]:
DROP VIEW IF EXISTS PessoasNumeroMedicamentos;

CREATE VIEW PessoasNumeroMedicamentos AS
SELECT UM.IdPessoa, COUNT(UM.CodMedicamento) Numero
FROM UsoMedicamento UM
GROUP BY IdPessoa;

SELECT AVG(PNM.Numero)
FROM PessoasNumeroMedicamentos PNM

## 6) Pessoas que usam mais do que a média

* Liste o id das pessoas que usam mais medicamentos do que a média.

In [ ]:
SELECT PNM.IdPessoa, PNM.Numero
FROM PessoasNumeroMedicamentos PNM
WHERE PNM.Numero > (SELECT AVG(PNM.Numero)
                    FROM PessoasNumeroMedicamentos PNM)
ORDER BY PNM.Numero DESC;

## 7) Análise do uso de medicamento

* Considere que um médico quer fazer responder a seguinte questão: pessoas tendem a usar mais medicamentos conforme ficam mais velhas?
* Escreva uma query que ajude o médico a realizar esta análise.

In [ ]:
SELECT P.Idade, AVG(PNM.Numero)
FROM PessoasNumeroMedicamentos PNM, Pessoa P
WHERE P.Id = PNM.IdPessoa
GROUP BY P.Idade 
ORDER BY P.Idade;

# II. Análise de correlação em pares
## Estudo de ação cruzada entre medicamentos

* Um pesquisador quer estudar o efeito cruzado de usar dois medicamentos simultaneamente. 
* Para isso ele precisará da seguinte sequência de queries.

## 8) Medicamentos tomados em conjunto
* Construa uma VIEW com duas colunas: medicamentoA, medicamentoB
* Nessa VIEW, é criada uma tupla (medicamentoA, medicamentoB) toda vez que uma pessoa toma o medicamento A e também o medicamento B.
* Haverá repetição de tuplas, já que mais de uma pessoa pode tomar o mesmo par de medicamentos.
* Cuidado para não duplicar a mesma informação, ou seja, considerando que uma pessoa tomou o medicamentoX e o medicamentoY, cuidado para não criar duas tuplas: (medicamentoX, medicamentoY) e (medicamentoY, medicamentoX).

In [ ]:
DROP VIEW IF EXISTS MedicamentosConjunto;

CREATE VIEW MedicamentosConjunto AS
SELECT UM.CodMedicamento MedicamentoA, UMB.CodMedicamento MedicamentoB
FROM UsoMedicamento UM, (Select * FROM UsoMedicamento) UMB
WHERE UM.IdPessoa = UMB.IdPessoa AND UM.CodMedicamento > UMB.CodMedicamento;

SELECT * FROM MedicamentosConjunto;

## 9) Total de medicamento tomados em conjunto
* Apresente o total de pessoas que toma cada par de medicamentos.

In [ ]:
SELECT MC.MedicamentoA, MC.MedicamentoB, COUNT(*) TotalConjunta
FROM MedicamentosConjunto MC
GROUP BY MC.medicamentoA, MC.medicamentoB;

# III. Análise de correlação entre variáveis
* Consiste na análise da seguinte questão: como a alteração de uma variável afeta a outra.
## Matriz de análise
* Considere a matriz abaixo.

In [4]:
CREATE TABLE Matriz (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  IdadeAte60 BOOLEAN,
  MaisUmAnoMedicamento BOOLEAN,
  MedicamentosAcimaMedia BOOLEAN,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade,
    FALSE, FALSE, FALSE
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

SELECT * FROM Matriz;

## 10) Atualização simples da matriz
* Utilize o comando UPDATE para atualizar a coluna IdadeAte60 da tabela colocando verdadeiro para as pessoas que têm até 60 anos de idade.

In [ ]:
UPDATE Matriz M
SET IdadeAte60 = true
WHERE M.Idade < 61;

SELECT * FROM Matriz

## 11) Atualização vinculada a uma segunda tabela
* Utilize o comando UPDATE para atualizar a coluna MaisUmAnoMedicamento, colocando verdadeiro para aquelas pessoas que usam pelo menos um medicamento há mais de um ano.

In [ ]:
UPDATE Matriz M
SET M.MaisUmAnoMedicamento = true
WHERE M.Id IN (SELECT DISTINCT UM.IdPessoa
             FROM UsoMedicamento UM
             WHERE UM.DiasUso > 365);
             
SELECT * FROM Matriz;

## 12) Atualização baseada em uma consulta complexa
* Utilize o comando UPDATE para atualizar a coluna MedicamentosAcimaMedia, colocando verdadeiro para aquelas pessoas que usam um número de medicamentos acima da média.

In [ ]:
UPDATE Matriz M
SET M.MedicamentosAcimaMedia = true
WHERE M.Id IN (SELECT PNM.IdPessoa
               FROM PessoasNumeroMedicamentos PNM
               WHERE PNM.Numero > (SELECT AVG(PNM.Numero)
                                   FROM PessoasNumeroMedicamentos PNM));
               
SELECT * FROM Matriz;

## 13) Que análise interessante pode ser feita?
* A partir da Matriz criada, proponha uma análise interessante.

### 13a) Descreva a seguir (em Markdown) qual a análise que você propõe.

Vamos analisar se as pessoas mais velhas tomam mais remédios acima da média que os mais novos. Além disso, vamos ver para pessoas que tomam mais remédios também. Importante lembrar que pessoas abaixo de 60 anos também podem tomar remédios acima da média, a análise aqui é se a ocorrência nas pessoas acima de 60 anos é maior ou não. Dessa forma, vamos pegar o total de pessoas nos grupos abaixo de 60 anos e acima, e o total de pessoas nesses grupos que tomam mais remédios que a média, assim podemos comparar a porcetagem(ocorrência).

### 13b) Escreva uma consulta em SQL que fornece informações para a análise proposta.

In [ ]:
SELECT M.IdadeAte60, COUNT(*)
FROM Matriz M
GROUP BY M.IdadeAte60;

SELECT MM.IdadeAte60, COUNT(*)
FROM (SELECT M.IdadeAte60
      FROM Matriz M
      WHERE M.MedicamentosAcimaMedia = true) MM
GROUP BY MM.IdadeAte60;

SELECT MM.IdadeAte60, COUNT(*)
FROM (SELECT M.IdadeAte60
      FROM Matriz M
      WHERE M.MaisUmAnoMedicamento = true) MM
GROUP BY MM.IdadeAte60;